In [26]:
#Basic libraries
import pandas as pd 
import numpy as np 

#NLTK libraries
import nltk
import re
import string
from wordcloud import WordCloud,STOPWORDS


# Machine Learning libraries
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score, roc_auc_score,roc_curve
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import neighbors
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
#Visualization libraries
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Other miscellaneous libraries
from collections import Counter


In [2]:
toys_reviews7 = pd.read_csv('tag_nlp_final.csv')
toys_reviews7.head(2)

,Unnamed: 0,reviewerID,asin,overall,reviewTime,helpful_score,reviews,Sentiment_score,Sentiments
0,0,A1VXOAVRGKGEAK,0439893577,5,2014-01-29,0.0,like item price granddaughter want mark want l...,0.4767,Positive
1,1,A8R62G708TSCM,0439893577,4,2014-03-28,1.0,love magnet easel great move different areas w...,0.9545,Extreme Positive


In [3]:
x=toys_reviews7['reviews']
y=toys_reviews7['Sentiments']
trainx,testx,trainy,testy= train_test_split(x,y,test_size=0.4,random_state=20)
print("trainx={}, trainy={},testx={}, testy={}".format(len(trainx),len(trainy),len(testx),len(testy)))
      
vectorizer = TfidfVectorizer(sublinear_tf=True,
                             max_df=0.5,
                             min_df=1,
                             use_idf=True,
                             smooth_idf=True,lowercase=True, max_features=2500)

trainx_v=vectorizer.fit_transform(trainx)
testx_v=vectorizer.transform(testx)
print(trainx_v.shape)
print(testx_v.shape)

trainx=99999, trainy=99999,testx=66667, testy=66667
(99999, 2500)
(66667, 2500)


In [4]:
trainx_v=trainx_v.toarray()
testx_v=testx_v.toarray()

### Naive Bayes

In [5]:
nb = GaussianNB().fit(trainx_v,trainy)
scorenb=round(nb.score(trainx_v,trainy)*100,2)
y_prednb =nb.predict(testx_v)


In [6]:
def evaluate(actual,predicted):
    print('Accuracy:',accuracy_score(actual,predicted))
    print('------------------------------------------------------------')
    print('\n Confusion Matrix: \n',pd.crosstab(actual,predicted,margins=True))
    print('------------------------------------------------------------')
    print("\nclassification report: \n",classification_report(testy, predicted))

In [21]:
evaluate(testy,y_pred_dt)

Accuracy: 0.8701906490467548
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Positive  Positive    All
Sentiments                                         
Extreme Negative               565         0    565
Extreme Positive             57985        11  57996
Negative                      1170         0   1170
Neutral                       1422         0   1422
Positive                      5486        28   5514
All                          66628        39  66667
------------------------------------------------------------

classification report: 
                   precision    recall  f1-score   support

Extreme Negative       0.00      0.00      0.00       565
Extreme Positive       0.87      1.00      0.93     57996
        Negative       0.00      0.00      0.00      1170
         Neutral       0.00      0.00      0.00      1422
        Positive       0.72      0.01      0.01      5514

        accuracy                   

### Decision tree classifier

In [7]:
dtm = DecisionTreeClassifier(max_depth=6,min_samples_split=5,max_leaf_nodes=10)
dtm1=dtm.fit(trainx_v,trainy)
y_pred_dt=dtm1.predict(testx_v)


In [8]:
evaluate(testy,y_pred_dt)

Accuracy: 0.8701906490467548
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Positive  Positive    All
Sentiments                                         
Extreme Negative               565         0    565
Extreme Positive             57985        11  57996
Negative                      1170         0   1170
Neutral                       1422         0   1422
Positive                      5486        28   5514
All                          66628        39  66667
------------------------------------------------------------

classification report: 
                   precision    recall  f1-score   support

Extreme Negative       0.00      0.00      0.00       565
Extreme Positive       0.87      1.00      0.93     57996
        Negative       0.00      0.00      0.00      1170
         Neutral       0.00      0.00      0.00      1422
        Positive       0.72      0.01      0.01      5514

        accuracy                   

### Random forest classifier

In [9]:
modelrf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=4, n_estimators=100, oob_score=True).fit(trainx_v,trainy)
y_predr=modelrf.predict(testx_v)
evaluate(testy,y_predr)

Accuracy: 0.8699356503217484
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Positive    All
Sentiments                               
Extreme Negative               565    565
Extreme Positive             57996  57996
Negative                      1170   1170
Neutral                       1422   1422
Positive                      5514   5514
All                          66667  66667
------------------------------------------------------------

classification report: 
                   precision    recall  f1-score   support

Extreme Negative       0.00      0.00      0.00       565
Extreme Positive       0.87      1.00      0.93     57996
        Negative       0.00      0.00      0.00      1170
         Neutral       0.00      0.00      0.00      1422
        Positive       0.00      0.00      0.00      5514

        accuracy                           0.87     66667
       macro avg       0.17      0.20      0.19     66667

In [10]:
evaluate(testy,y_predr)

Accuracy: 0.8699356503217484
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Positive    All
Sentiments                               
Extreme Negative               565    565
Extreme Positive             57996  57996
Negative                      1170   1170
Neutral                       1422   1422
Positive                      5514   5514
All                          66667  66667
------------------------------------------------------------

classification report: 
                   precision    recall  f1-score   support

Extreme Negative       0.00      0.00      0.00       565
Extreme Positive       0.87      1.00      0.93     57996
        Negative       0.00      0.00      0.00      1170
         Neutral       0.00      0.00      0.00      1422
        Positive       0.00      0.00      0.00      5514

        accuracy                           0.87     66667
       macro avg       0.17      0.20      0.19     66667

### Logistic cv

In [11]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,multi_class='multinomial').fit(trainx_v,trainy)
y_pred_lrcv=clf.predict(testx_v)

In [12]:
evaluate(testy,y_pred_lrcv)

Accuracy: 0.9116504417477913
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Negative  Extreme Positive  Negative  Neutral  \
Sentiments                                                                
Extreme Negative               179                60       213       64   
Extreme Positive                36             57010        52       63   
Negative                       132               114       344      292   
Neutral                         43               155       180      324   
Positive                        57              2076       192      269   
All                            447             59415       981     1012   

col_0             Positive    All  
Sentiments                         
Extreme Negative        49    565  
Extreme Positive       835  57996  
Negative               288   1170  
Neutral                720   1422  
Positive              2920   5514  
All                   4812  66667  

### adaboost

In [13]:
ada = AdaBoostClassifier(n_estimators=40, learning_rate=1).fit(trainx_v,trainy)
y_pred_ada=ada.predict(testx_v)

In [14]:
evaluate(testy,y_pred_ada)

Accuracy: 0.8759356203218984
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Negative  Extreme Positive  Negative  Neutral  \
Sentiments                                                                
Extreme Negative                63               263        80        0   
Extreme Positive                45             57312        29        1   
Negative                        45               530        92        0   
Neutral                         21               726        35        0   
Positive                        23              4519        43        0   
All                            197             63350       279        1   

col_0             Positive    All  
Sentiments                         
Extreme Negative       159    565  
Extreme Positive       609  57996  
Negative               503   1170  
Neutral                640   1422  
Positive               929   5514  
All                   2840  66667  

### Logistic one vs rest

In [15]:
m1=LogisticRegression(max_iter=100)
m1_ova=OneVsRestClassifier(m1).fit(trainx_v,trainy)

In [16]:
p1=m1_ova.predict(testx_v)
pd.DataFrame({'Actual':testy,"predicted":p1})[1045:1050]

,Actual,predicted
62006,Extreme Positive,Extreme Positive
107212,Extreme Positive,Extreme Positive
160964,Positive,Positive
41830,Extreme Positive,Extreme Positive
139048,Negative,Positive


In [17]:
evaluate(testy,p1)

Accuracy: 0.8909355453222734
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Negative  Extreme Positive  Negative  Neutral  \
Sentiments                                                                
Extreme Negative                45               150        74        7   
Extreme Positive                 0             57808         7        1   
Negative                         7               298        59       19   
Neutral                          2               417        20        7   
Positive                         7              3996        23       11   
All                             61             62669       183       45   

col_0             Positive    All  
Sentiments                         
Extreme Negative       289    565  
Extreme Positive       180  57996  
Negative               787   1170  
Neutral                976   1422  
Positive              1477   5514  
All                   3709  66667  

### multinomial

In [18]:
m2 = MultinomialNB()
ovr2=OneVsRestClassifier(m2).fit(trainx_v,trainy)

In [19]:
p2=ovr2.predict(testx_v)
pd.DataFrame({'Actual':testy,"predicted":p2})[:10]

,Actual,predicted
24794,Positive,Extreme Positive
5357,Extreme Positive,Extreme Positive
18023,Extreme Positive,Extreme Positive
123757,Extreme Positive,Extreme Positive
4199,Extreme Positive,Extreme Positive
19774,Positive,Extreme Positive
94890,Extreme Positive,Extreme Positive
50504,Extreme Positive,Extreme Positive
78830,Extreme Positive,Extreme Positive
156080,Negative,Extreme Positive


In [20]:
evaluate(testy,p2)

Accuracy: 0.870040649796751
------------------------------------------------------------

 Confusion Matrix: 
 col_0             Extreme Negative  Extreme Positive  Negative  Positive  \
Sentiments                                                                 
Extreme Negative                11               540         4        10   
Extreme Positive                 5             57987         0         4   
Negative                         1              1164         0         5   
Neutral                          0              1420         0         2   
Positive                         1              5508         0         5   
All                             18             66619         4        26   

col_0               All  
Sentiments               
Extreme Negative    565  
Extreme Positive  57996  
Negative           1170  
Neutral            1422  
Positive           5514  
All               66667  
------------------------------------------------------------

classifica